In [80]:
from PyNite import FEModel3D
from PyNite.Visualization import render_model
from math import sqrt
from math import tan
from math import radians

In [91]:
def frame_displ_y(H, L, w, ang_d):
    """
    H, l, w, ang_d
    """
    
    frame_model = FEModel3D() # Creates an empty model
    H  # mm
    L  # mm
    w  # N/mm
    ang = radians (ang_d) # rad

    #Add nodes
    frame_model.add_node("node1",0,0,0)
    frame_model.add_node("node2",0,H,0)
    frame_model.add_node("node3",L/2,H+((L/2)*tan(ang)),0)
    frame_model.add_node("node4",L,H,0)
    frame_model.add_node("node5",L,0,0)

    m = (L/2)**2 + 500**2
    member_length = sqrt(m)

    #Define material
    I_1 = 100e6
    I_2 = 800e6
    J = 100e3
    A = 2000
    frame_model.add_material(name='Steel', E=200e3, G=77e3, nu=0.3, rho=7.7e-6)

    #Add frame members
    frame_model.add_member('M1', 'node1', 'node2', 'Steel', I_1, I_2, J, A)
    frame_model.add_member('M2', 'node2', 'node3', 'Steel', I_1, I_2, J, A)
    frame_model.add_member('M3', 'node3', 'node4', 'Steel', I_1, I_2, J, A)
    frame_model.add_member('M4', 'node4', 'node5', 'Steel', I_1, I_2, J, A)

    #Add load
    frame_model.add_load_combo('L',{"D":1})
    frame_model.add_member_dist_load("M1", "Fy", -w, -w, 0, H, case="D")
    frame_model.add_member_dist_load("M4", "Fy", w, w, 0, H, case="D")
    frame_model.add_member_dist_load("M2", "Fy", -w, -w, 0, member_length, case="D")
    frame_model.add_member_dist_load("M3", "Fy", -w, -w, 0, member_length, case="D")

    #Add boundary condition
    frame_model.def_support('node1', True, True, True, True, True, True)
    frame_model.def_support('node5', True, True, True, True, True, True)
    
    #render_model(frame_model, combo_name="L", annotation_size=300)
    #Submit for analysis
    frame_model.analyze()

    #Displacement (not finished)
    displ_y_1 = frame_model.Members['M1'].min_deflection('dy', combo_name="L")
    displ_y_2 = frame_model.Members['M2'].min_deflection('dy', combo_name="L")
    displ_y_3 = frame_model.Members['M3'].min_deflection('dy', combo_name="L")
    displ_y_4 = frame_model.Members['M4'].min_deflection('dy', combo_name="L")
    displ = min(displ_y_1,displ_y_2, displ_y_3, displ_y_4)

    #max momemnt
    moment_max_1=abs(frame_model.Members['M1'].max_moment('Mz', combo_name = 'L')/10**6)
    moment_max_2=abs(frame_model.Members['M2'].max_moment('Mz', combo_name = 'L')/10**6)
    moment_max_3=abs(frame_model.Members['M3'].max_moment('Mz', combo_name = 'L')/10**6)
    moment_max_4=abs(frame_model.Members['M4'].max_moment('Mz', combo_name = 'L')/10**6)
    #min moment
    moment_min_1=abs(frame_model.Members['M1'].min_moment('Mz', combo_name = 'L')/10**6)
    moment_min_2=abs(frame_model.Members['M2'].min_moment('Mz', combo_name = 'L')/10**6)
    moment_min_3=abs(frame_model.Members['M3'].min_moment('Mz', combo_name = 'L')/10**6)
    moment_min_4=abs(frame_model.Members['M4'].min_moment('Mz', combo_name = 'L')/10**6)
    moment = max (moment_max_1, moment_max_2, moment_max_3, moment_max_4, moment_min_1, moment_min_2, moment_min_3, moment_min_4)

    #max shear
    shear_max_1=abs(frame_model.Members['M1'].max_shear('Fy', combo_name = 'L')/10**3)
    shear_max_2=abs(frame_model.Members['M2'].max_shear('Fy', combo_name = 'L')/10**3)
    shear_max_3=abs(frame_model.Members['M3'].max_shear('Fy', combo_name = 'L')/10**3)
    shear_max_4=abs(frame_model.Members['M4'].max_shear('Fy', combo_name = 'L')/10**3)
    #min shear
    shear_min_1=abs(frame_model.Members['M1'].min_shear('Fy', combo_name = 'L')/10**3)
    shear_min_2=abs(frame_model.Members['M2'].min_shear('Fy', combo_name = 'L')/10**3)
    shear_min_3=abs(frame_model.Members['M3'].min_shear('Fy', combo_name = 'L')/10**3)
    shear_min_4=abs(frame_model.Members['M4'].min_shear('Fy', combo_name = 'L')/10**3)
    shear = max(shear_max_1, shear_max_2, shear_max_3, shear_max_4, shear_min_1, shear_min_2, shear_min_3, shear_min_4)
    
    return moment

In [96]:
frame_displ_y(3000, 10000, 12, 15)

75.00987382939405

In [99]:
for L in range (10000, 10200):
    ddd = frame_displ_y(3000, L, 12, 15)
    print (ddd)

75.00987382939405
75.02145561724605
75.0330382059145
75.04462159530573
75.05620578532731
75.0677907758853
75.07937656688672
75.0909631582381
75.10255054984648
75.1141387416183
75.12572773346069
75.13731752527997
75.14890811698343
75.16049950847726
75.1720916996688
75.18368469046442
75.19527848077104
75.20687307049553
75.2184684595447
75.23006464782523
75.24166163524397
75.2532594217082
75.26485800712393
75.27645739139828
75.28805757443824
75.29965855615058
75.31126033644244
75.32286291522048
75.33446629239104
75.34607046786171
75.35767544153887
75.36928121332967
75.3808877831409
75.39249515087965
75.40410331645248
75.41571227976657
75.42732204072844
75.43893259924548
75.45054395522435
75.46215610857193
75.47376905919515
75.48538280700119
75.4969973518964
75.50861269378834
75.52022883258373
75.53184576818971
75.54346350051289
75.55508202946025
75.56670135493911
75.57832147685613
75.58994239511875
75.60156410963312
75.61318662030726
75.62480992704718
75.63643402976037
75.64805892835415
7

In [100]:
L = 10000
M = 0
while M <= 86:
    M = frame_displ_y(3000, L, 12, 15)
    L+=1
print (L, M)
    

10922 86.00404411448042


In [ ]:
H = 3000
L = 1000 
ang = 45
ang = radians(ang)
tan(ang)
#((L/2)*tan(ang))
#H+((L/2)*tan(ang))
   